In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.dummy import DummyClassifier
from sklearn.preprocessing import RobustScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline, FunctionTransformer
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pickle


In [6]:
data = pd.read_csv('../raw_data/data.csv')
data.head(2)

,R_fighter,B_fighter,Referee,date,location,Winner,title_bout,weight_class,B_avg_KD,B_avg_opp_KD,...,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
0,Adrian Yanez,Gustavo Lopez,Chris Tognoni,2021-03-20,"Las Vegas, Nevada, USA",Red,False,Bantamweight,0.0,0.0,...,0,1,0,0,Orthodox,170.18,177.80,135.0,31.0,27.0
1,Trevin Giles,Roman Dolidze,Herb Dean,2021-03-20,"Las Vegas, Nevada, USA",Red,False,Middleweight,0.5,0.0,...,0,3,0,0,Orthodox,182.88,187.96,185.0,32.0,28.0


In [7]:
X = data.drop(['Winner'], axis=1)
y= data.Winner

In [8]:
#It's a placeholder for future model selection
#X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, 
#                                                       train_size=0.8, 
#                                                       random_state=42, stratify = y)

In [9]:
num_preproc = Pipeline([
    ("num_imputer", SimpleImputer(strategy = "median")),
    ("scaler", RobustScaler())
])

In [10]:
cat_preproc = Pipeline([
    ("cat_imputer", SimpleImputer(strategy = "constant", fill_value="Unknown"))
])

In [11]:
preproc = ColumnTransformer([
    ("num_tr", num_preproc, make_column_selector(dtype_include = ["float64", "int64"])),
    ("cat_tr", cat_preproc, make_column_selector(dtype_include = ["object"]))
])

In [12]:
base_line_pipe = Pipeline([
    ("preproc", preproc),
    ("dummy_classifier", DummyClassifier(strategy='most_frequent'))
])
    
base_line_pipe

Pipeline(steps=[('preproc',
                 ColumnTransformer(transformers=[('num_tr',
                                                  Pipeline(steps=[('num_imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   RobustScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7fb39ac30d10>),
                                                 ('cat_tr',
                                                  Pipeline(steps=[('cat_imputer',
                                                                   SimpleImputer(fill_value='Unknown',
                                                                                 strategy='constant'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7fb39ac33c80>)])),
                ('dummy_classifier',
                 DummyClassifier(strategy='most_frequent'))])

In [14]:
 #Didn't use train_test_split for BASE_model. It's dummy_classifier with constant answer
base_line_pipe.fit(X,y)
y_pred = base_line_pipe.predict(X)

In [15]:
base_model_accuracy = accuracy_score(y, y_pred)
base_model_accuracy

0.6618429807052562

In [16]:
#Export the fitted pipeline as a pickle file
with open('../models/base_model_acc066.pkl', 'wb') as file:
    pickle.dump(base_line_pipe, file)
print("Base_line_pipe is successfully saved as 'base_model_acc066.pkl'")

Base_line_pipe is successfully saved as 'base_model_acc066.pkl'
